In [ ]:
# imports

import os
import re
import math
import random
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from huggingface_hub import login
from tqdm import tqdm
import matplotlib.pyplot as plt
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer

In [ ]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
%matplotlib inline

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"

In [ ]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Electronics", split="full", trust_remote_code=True)

In [ ]:
print(f"There are {len(dataset):,} items in the dataset")
print("Here is the first:")
item = dataset[0]
print(item['title'])
print(item['description'])
print(item['features'])
print(item['price'])

In [ ]:
class Item:

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

    def __init__(self, data):
        self.title = data['title']
        self.description = self.clean(data['description'])
        self.features = self.clean(data['features'])
        self.price_str = data['price']
        self.price = float(self.price_str)
        self._token_count = None

    def clean(self, details):
        result = ' '.join(details)
        return re.sub(r'[\[\]【】\s]+', ' ', result).strip()

    def question(self):
        prompt = "How much does this cost?\n"
        prompt += f"Title: {self.title}\n"
        prompt += f"Description: {self.description}\n"
        prompt += f"Features: {self.features}\n"
        return prompt

    def inference_prompt(self):
        return f"{self.question()}Answer: $"

    def train_prompt(self):
        return f"{self.inference_prompt()}{self.price_str}"

    def token_count(self):
        if self._token_count == None:
            self._token_count = len(self.tokenizer.encode(self.train_prompt()))
        return self._token_count

    def tokens_between(self, low, high):
        token_count = self.token_count()
        return token_count >= low and token_count < high

In [ ]:
# Create a list called "items" with all our datapoints that have a valid price

items = []
for data in tqdm(dataset):
    try:
        if float(data['price']) > 0:
            items.append(Item(data))
    except ValueError:
        pass

In [ ]:
# Let's investigate:

print(f"There are {len(items):,} out of {len(dataset):,} with prices\n")
print(f"Item 0 has {items[0].token_count()} tokens:\n")
print(items[0].train_prompt())
print(f"\nItem 1 has {items[1].token_count()} tokens:\n")
print(items[1].train_prompt())

In [ ]:
# Plot the distribution of character count

lengths = [len(item.train_prompt()) for item in items]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Length')
ax.set_ylabel('Count of items');
_ = ax.hist(lengths, rwidth=0.7, color="lightblue", bins=range(0, 5000, 250))

print(f"Average length is {sum(lengths)/len(lengths):,.1f} and highest length is {max(lengths):,}\n")

In [ ]:
print(f"There are total {len(items):,} items")
cutoff = 1200
selection = [item for item in items if len(item.train_prompt()) < cutoff]
print(f"There are total {len(selection):,} with under {cutoff:,} character training prompt")

In [ ]:
# Calculate token sizes in selection

token_counts = [item.token_count() for item in tqdm(selection)]

In [ ]:
# Plot the distribution of tokens

fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="orange", bins=range(0, 500, 25))

In [ ]:
# Let's limit our dataset to documents with 60-180 tokens

low_cutoff = 60
high_cutoff = 180
subset = [item for item in tqdm(selection) if item.tokens_between(low_cutoff, high_cutoff)]
subset_count = len(subset)
count = len(items)
print(f"\nBetween {low_cutoff} and {high_cutoff}, we get {subset_count:,} out of {count:,} which is {subset_count/count*100:.1f}%")

In [ ]:
# Plot the distribution again to check it looks as expected

token_counts = [item.token_count() for item in subset]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="purple", bins=range(0, 300, 10))

In [ ]:
# Plot the distribution of prices

prices = [float(item.price) for item in subset]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Price ($)')
ax.set_ylabel('Count of items');
_ = ax.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 500, 20))

print(f"Average price is ${sum(prices)/len(prices):.2f} and highest price is ${max(prices):,.2f}\n")

In [ ]:
# Pick the most expensive 30,000 items, then pick 12,000 of the next 20,000

random.seed(42)
sorted_subset = sorted(subset, key=lambda item: item.price, reverse=True)
top_30k = sorted_subset[:30000]
other_12k = random.sample(sorted_subset[30000:50000], k=12000)
sample = top_30k + other_12k
print(len(sample))

In [ ]:
# Plot the distribution of prices

prices = [float(item.price) for item in sample]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Price ($)')
ax.set_ylabel('Count of items');
_ = ax.hist(prices, rwidth=0.7, color="orange", bins=range(0, 500, 20))

print(f"Average price is ${sum(prices)/len(prices):.2f} and highest price is ${max(prices):,.2f}\n")

In [ ]:
sizes = [len(item.train_prompt()) for item in sample]
prices = [item.price for item in sample]

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(sizes, prices, s=2, color="red")

# Add labels and title
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Item Price vs Size')

# Display the plot
plt.show()

In [ ]:
# Plot the distribution again to check it looks as expected

token_counts = [item.token_count() for item in subset]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="purple", bins=range(0, 300, 10))

In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:40000]
test = sample[40000:]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
average = sum(t.price for t in train)/len(train)
average

In [ ]:
class TestRunner:

    def __init__(self, predictor, data, size=None):
        self.predictor = predictor
        self.data = data
        self.size = size or len(data)
        self.guesses = []
        self.truths = []
        self.errors = []

    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        color = RED if error>=20 else YELLOW if error>=10 else GREEN
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40]+"..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        print(f"{color}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} Item: {title}{RESET}")

    def chart(self):
        max_error = max(self.errors)
        colors = [(max_error - error)**3 for error in self.errors]
        plt.figure(figsize=(10, 6))
        plt.scatter(self.truths, self.guesses, s=2, c=colors, cmap='RdYlGn')
        plt.xlabel('Truth')
        plt.ylabel('Guess')
        plt.title('Guess vs Truth')
        plt.show()

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        average_error = sum(self.errors) / self.size
        print(f"Average Error = ${average_error:,.2f}")
        hits = [e for e in self.errors if e<10]
        print(f"Hit rate = {len(hits)/self.size*100:.1f}%")
        self.chart()

In [ ]:
def flat_predictor(item):
    return 218.28366025006002

In [ ]:
TestRunner(flat_predictor, test, 100).run()

In [ ]:
stop = set(['the', 'and', 'for', 'is', 'to', 'this', 'with', 'a', 'of', 'your', 'are', 'in','from', 'you', 'or', 'an'])

def words(item):
  text = f"{item.title} {item.description} {item.features}"
  text = re.sub(r'[()\[\]{},\'"-]', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  words = text.strip().lower().split(' ')
  filtered = [word for word in words if word not in stop]
  return " ".join(filtered)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np

np.random.seed(42)

documents = [words(item) for item in train]
labels = np.array([float(item.price) for item in train])

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

regressor = LinearRegression()
regressor.fit(X, labels)

In [ ]:
def linear_regression_predictor(item):
    np.random.seed(42)
    x = vectorizer.transform([words(item)])
    return max(regressor.predict(x)[0], 0)

In [ ]:
TestRunner(linear_regression_predictor, test, 100).run()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR

np.random.seed(42)

documents = [words(item) for item in train]
labels = np.array([float(item.price) for item in train])

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

regressor = SVR(kernel='linear')
regressor.fit(X, labels)

In [ ]:
def svr_predictor(item):
    np.random.seed(42)
    x = vectorizer.transform([words(item)])
    return max(regressor.predict(x)[0], 0)

In [ ]:
TestRunner(svr_predictor, test, 100).run()

In [ ]:
def messages_for(item):
    system_message = "You predict prices based on a description. Reply only with the price in $, no explanation or comments"
    user_prompt = item.question()
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

In [ ]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
def gpt_predictor(item):
    response = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for(item),
        seed=42
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
TestRunner(gpt_predictor, test, 100).run()

In [ ]:
test[0].train_prompt()

In [ ]:
train_prompts = [item.train_prompt() for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.inference_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
# Create a Dataset from the lists
train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
DATASET_NAME = "ed-donner/electronics"
dataset.push_to_hub(DATASET_NAME, private=True)